In [7]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import sklearn
import os
import PyPDF2
import pdfplumber
import fitz
import re

In [14]:
from concurrent.futures import ThreadPoolExecutor

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file using pdfplumber."""
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def preprocess_text(text):
    """Clean the extracted text for easier processing."""
    text = re.sub(r'\n+', '\n', text)  # Remplacer les sauts de ligne multiples par un seul
    text = re.sub(r'\s+', ' ', text)  # Remplacer les espaces multiples par un seul
    return text

def extract_information(text):
    """Attempt to extract title, author, date, and other info from the text."""
    # Rechercher les sections du texte où se trouvent les informations
    title = re.search(r"(Title|Titre):?\s*(.*)", text, re.IGNORECASE)
    if title:
        title = title.group(2)
    else:
        # Si la première méthode ne fonctionne pas, on essaie de prendre la première ligne
        title = text.split('\n')[0][:100]  # Prendre les premiers 100 caractères comme titre

    author = re.search(r"(Author|Auteur):?\s*(.*)", text, re.IGNORECASE)
    if author:
        author = author.group(2)
    else:
        author = "Auteur non trouvé"

    # Extraction de la date (en supposant un format standard comme YYYY-MM-DD, DD/MM/YYYY, etc.)
    date = re.search(r"(\b\d{4}[-/]\d{2}[-/]\d{2}\b|\b\d{2}[-/]\d{2}[-/]\d{4}\b)", text)
    if date:
        date = date.group(0)
    else:
        date = "Date non trouvée"

    # Extraction du contenu après les premières informations trouvées
    content_start = text.find(title) + len(title) if title else 0
    content = text[content_start:].strip()

    return {
        "title": title.strip(),
        "author": author.strip(),
        "date": date.strip(),
        "content": content
    }

def process_single_pdf(pdf_file, pdf_folder):
    pdf_path = os.path.join(pdf_folder, pdf_file)
    text = extract_text_from_pdf(pdf_path)
    cleaned_text = preprocess_text(text)
    info = extract_information(cleaned_text)
    info["file_name"] = pdf_file  # Ajouter le nom du fichier pour référence
    return info

def process_pdfs(pdf_folder):
    pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith(".pdf")]
    results = []

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_single_pdf, pdf_file, pdf_folder) for pdf_file in pdf_files]
        for future in futures:
            results.append(future.result())

    # Créer un DataFrame à partir des résultats
    df = pd.DataFrame(results)
    return df

pdf_folder_path = "articles et bouquins"
pdf_data_df = process_pdfs(pdf_folder_path)


In [15]:
# Afficher les résultats
pdf_data_df.head()

,title,author,date,content,file_name
0,Une am´elioration de la convergence de la m´et...,Email addresses: fatimabouyghf3@gmail.com(F.BO...,Date non trouvée,"LabMIA-SI, University of Mohammed V Rabat, Mor...",Article_11.pdf
1,L’am´elioration de la convergence de la m´etho...,Email address: fatimabouyghf3@gmail.com(F.BOUY...,Date non trouvée,e d’un seul ou plusieurs second membres F. BOU...,Article_22.pdf
2,Une approche unifi´ee pour les m´ethodes de so...,Email addresses: fatimabouyghf3@gmail.com(F.BO...,Date non trouvée,"lin´eaires F. BOUYGHFa,b, A. MESSAOUDIa, H. SA...",Article_33.pdf
3,1 Appendix ThecoefficientmatrixAandtheright–ha...,Auteur non trouvé,Date non trouvée,onjugate GradientMethod” by Hao Ji and Yaohang...,BFBCGappendix.pdf
4,Electronic Transactions on Numerical Analysis....,Auteur non trouvé,Date non trouvée,"ity Copyright2009, Kent State University. htt...",bloc sadok messaoudi.pdf


In [16]:
# Exporter les données vers un fichier CSV si nécessaire
pdf_data_df.to_csv("corpus.csv", index=False)